# Import useful libraries

In [8]:
import numpy as np

In [13]:
import pandas as pd

In [12]:
import matplotlib.pyplot as plt

In [41]:
from keras.preprocessing import image

Using TensorFlow backend.


In [22]:
from pathlib import Path

# Data Loading and Cleaning

In [249]:
labels_converter = {'Pikachu':0, 'Bulbasaur':1, 'Charmander':2}
rev_labels = {0:'Pikachu', 1:'Bulbasaur', 2:'Charmander'}

image_data = {}

p = Path('Images')

for d in p.glob('*.jpg'):
    data = image.load_img(d,target_size=(100,100))
    image_data.setdefault(str(d).split('\\')[1], np.array(data))


C:\Users\rajni\Anaconda3\envs\tensorflow\lib\site-packages\PIL\Image.py:965: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


In [84]:
pokemon = pd.read_csv('train.csv')

In [269]:
pdata = np.array(pokemon)

In [137]:
X_ = [] #input data
Y_ = [] #input labels
for i in pdata[:,0]:
    
    #store image pixel data in X
    temp = np.array(image_data.get(i))
    X_.append(temp.flatten())
    
for j in pdata[:,1]:
    
    #store labels in Y
    Y_.append(labels_converter.get(j))
    
X = np.array(X_)
Y = np.array(Y_)   
    
    

 # KNN Classifier model

In [295]:
def distance(a, b):
    '''Helper function for getting distance b/w 2 pts.
    '''
    return np.sum((a-b)**2)**.5

def KNN(X, Y, test_input, k=5):
    '''K-Nearest-Neighbours algorithm implementation
    '''
    
    flattened_input = test_input.flatten()
    iter_range = X.shape[0]
    dist_list = []
    
    for i in range(iter_range):
        
        dist = distance(flattened_input, X[i])
        dist_list.append((dist, Y[i]))
    
    sorted_dist_list = np.array(sorted(dist_list), dtype = 'int')
    n_neigh = np.unique(sorted_dist_list[:k,1], return_counts = 'True')
    index = np.argmax(n_neigh[1])
    
    #however, most weight is given to that element, who's distance from
    #test_input is zero
    for i in range(sorted_dist_list.shape[0]):
        if sorted_dist_list[i][0] == 0:
            index = i
            
    return rev_labels.get(sorted_dist_list[index][1])

# Test Phase

In [298]:
test_csv = pd.read_csv('test.csv')
test_array = np.array(test_csv)

#Create submission file as submissionn.csv
f = open('submission.csv', 'a')
f.write('ImageId,NameOfPokemon\n')


#load data from image_data to test_imaage_data
test_image_data = []
test_image_name = []

for i in test_array[:,0]:
    test_image_data.append(image_data.get(i))
    test_image_name.append(i)

#finally, write everything in the submission file, and then close the file    
for i in range(len(test_image_data)):
    f.write(test_image_name[i] + ',' + KNN(X, Y, test_image_data[i], 2) + '\n')
    
f.close()    

In [297]:
test_image = np.array(image.load_img('Images\\2031.jpg', target_size = (100,100)))

a = KNN(X, Y, test_image)
print(a)

Bulbasaur
